In [2]:
# filter dataset
!python3 MT-Preparation/filtering/filter.py ./en-zh.zh ./en-zh.en zh en

Dataframe shape (rows, columns): (231267, 2)
--- Rows with Empty Cells Deleted	--> Rows: 231267
--- Duplicates Deleted			--> Rows: 229646
--- Source-Copied Rows Deleted		--> Rows: 229640
--- Too Long Source/Target Deleted	--> Rows: 3088
--- HTML Removed			--> Rows: 3088
--- Rows will remain true-cased		--> Rows: 3088
--- Rows with Empty Cells Deleted	--> Rows: 3088
--- Rows Shuffled			--> Rows: 3088
--- Source Saved: ./en-zh.zh-filtered.zh
--- Target Saved: ./en-zh.en-filtered.en


In [4]:
# train a sentencepiece model for subwording
!python3 MT-Preparation/subwording/1-train_unigram.py ./en-zh.zh-filtered.zh ./en-zh.en-filtered.en

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=./en-zh.zh-filtered.zh --model_prefix=source --vocab_size=50000 --hard_vocab_limit=false --split_digits=true
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./en-zh.zh-filtered.zh
  input_format: 
  model_prefix: source
  model_type: UNIGRAM
  vocab_size: 50000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 0
  use_all_vocab: 

In [5]:
# train a SentencePiece model for subword tokenization
!python3 MT-Preparation/subwording/1-train_unigram.py ./en-zh.zh-filtered.zh ./en-zh.en-filtered.en

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=./en-zh.zh-filtered.zh --model_prefix=source --vocab_size=50000 --hard_vocab_limit=false --split_digits=true
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./en-zh.zh-filtered.zh
  input_format: 
  model_prefix: source
  model_type: UNIGRAM
  vocab_size: 50000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 0
  use_all_vocab: 

In [6]:
# subword the dataset
!python3 MT-Preparation/subwording/2-subword.py source.model target.model ./en-zh.zh-filtered.zh ./en-zh.en-filtered.en

Source Model: source.model
Target Model: target.model
Source Dataset: ./en-zh.zh-filtered.zh
Target Dataset: ./en-zh.en-filtered.en
Done subwording the source file! Output: ./en-zh.zh-filtered.zh.subword
Done subwording the target file! Output: ./en-zh.en-filtered.en.subword


In [7]:
# first 3 lines before subwording
!head -n 3 ./en-zh.zh-filtered.zh && echo "-----" && head -n 3 ./en-zh.en-filtered.en

它正在发生着。
43次。
因为我们已经看到了PS 234 的项目是怎么做的。
-----
It's happening.
Forty-three.
We saw how PS 234 worked.


In [8]:
# first 3 lines after subwording
!head -n 3 ./en-zh.zh-filtered.zh.subword && echo "---" && head -n 3 ./en-zh.en-filtered.en.subwordafter

▁它 正在 发生 着 。
▁ 4 3 次 。
▁因为 我们已经 看到了 P S ▁ 2 3 4 ▁ 的项目 是怎么做 的 。
---
head: ./en-zh.en-filtered.en.subwordafter: No such file or directory


In [10]:
# split the dataset into training set, development set, and test set
# Development and test sets should be between 100 and 500 segments (here we chose 200)
!python3 MT-Preparation/train_dev_split/train_dev_test_split.py 200 200 ./en-zh.zh-filtered.zh.subword ./en-zh.en-filtered.en.subword

Dataframe shape: (3088, 2)
--- Empty Cells Deleted --> Rows: 3088
--- Wrote Files
Done!
Output files
./en-zh.zh-filtered.zh.subword.train
./en-zh.en-filtered.en.subword.train
./en-zh.zh-filtered.zh.subword.dev
./en-zh.en-filtered.en.subword.dev
./en-zh.zh-filtered.zh.subword.test
./en-zh.en-filtered.en.subword.test


In [ ]:
!wc -l ./*.subword.*

     200 ./en-zh.en-filtered.en.subword.dev
     200 ./en-zh.en-filtered.en.subword.test
    2688 ./en-zh.en-filtered.en.subword.train
     200 ./en-zh.zh-filtered.zh.subword.dev
     200 ./en-zh.zh-filtered.zh.subword.test
    2688 ./en-zh.zh-filtered.zh.subword.train
    6176 total


In [14]:
# check the first and last line from each dataset
!echo "---First line---"
!head -n 1 ./*.{train,dev,test}

!echo -e "\n---Last line---"
!tail -n 1 ./*.{train,dev,test}

---First line---
==> ./en-zh.en-filtered.en.subword.train <==
▁It ' s ▁happening .

==> ./en-zh.zh-filtered.zh.subword.train <==
▁它 正在 发生 着 。

==> ./en-zh.en-filtered.en.subword.dev <==
▁We ▁call ▁it ▁O ME GA , ▁wh ich ▁is ▁an ▁ac r onym ▁for ▁Off shor e ▁Mem bran e ▁Enc los ures ▁for ▁Gr owing ▁Alg a e .

==> ./en-zh.zh-filtered.zh.subword.dev <==
▁我们 称 之 为 O ME GA ▁是 O ff s hore ▁Me mbr ane ▁Enc lo sure s ▁f or ▁G row ing ▁Al ga e ▁ 的 缩 写

==> ./en-zh.en-filtered.en.subword.test <==
▁S ign ▁language .

==> ./en-zh.zh-filtered.zh.subword.test <==
▁ 手 语 。

---Last line---
==> ./en-zh.en-filtered.en.subword.train <==
▁Miss !

==> ./en-zh.zh-filtered.zh.subword.train <==
▁“ 老师 !”

==> ./en-zh.en-filtered.en.subword.dev <==
▁All ▁right , ▁next .

==> ./en-zh.zh-filtered.zh.subword.dev <==
▁好 , ▁下一个 。

==> ./en-zh.en-filtered.en.subword.test <==
▁Yes , ▁M ama .

==> ./en-zh.zh-filtered.zh.subword.test <==
▁好的 , 妈妈
